In [1]:
# Imports
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score


In [2]:
df_cpi = pd.read_csv("data/cpi.csv")
df_interest = pd.read_csv("data/interest.csv")
df_rent_index = pd.read_csv("data/rentIndex.csv")
df_vacant = pd.read_csv("data/vacant.csv")
df_properties = pd.read_csv("data/properties.csv")
df_geo_attributes = pd.read_csv("data/geo_attributes.csv")
df_test = pd.read_csv("data/test.csv")
df_train = pd.read_csv("data/train.csv")


In [3]:
df_cpi.head()


,Data Series,CPI
0,2022 Dec,111.186
1,2022 Nov,110.959
2,2022 Oct,109.893
3,2022 Sep,110.339
4,2022 Aug,109.863


In [4]:
df_interest.head()


,Data Series,InterestRate
0,2022 Dec,1.48
1,2022 Nov,1.47
2,2022 Oct,1.17
3,2022 Sep,1.03
4,2022 Aug,0.95


In [5]:
df_rent_index.head()


,Data Series,RentIndex
0,2022 4Q,148.1
1,2022 3Q,137.9
2,2022 2Q,127.0
3,2022 1Q,119.0
4,2021 4Q,114.2


In [6]:
df_vacant.head()


,Data Series,Available,Vacant
0,2022 4Q,34084,524
1,2022 3Q,34084,514
2,2022 2Q,34084,452
3,2022 1Q,34084,517
4,2021 4Q,34084,576


In [7]:
df_properties.head()


,area,floorRange,propertyType,district,typeOfArea,tenure,street,project,marketSegment,property_key
0,226.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-7b6c69000
1,194.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-320ed2726
2,348.0,06-10,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-2bc5c4951
3,223.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-f060c5be0
4,195.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-a8d1004a9


In [8]:
df_geo_attributes.head()


,street,project,district,lat,lng,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m
0,LIM AH WOO ROAD,SUITES @ GUILLEMARD,15,1.312520,103.891760,7,2,3
1,HOLLAND ROAD,LOFT@HOLLAND,10,1.312364,103.797169,0,3,4
2,SOUTH BUONA VISTA ROAD,VIVA VISTA,5,1.279803,103.785910,0,0,2
3,UPPER SERANGOON ROAD,PARK RESIDENCES KOVAN,19,1.357934,103.881932,7,2,1
4,NATHAN ROAD,LOFT @ NATHAN,10,1.294461,103.827621,2,4,4


In [9]:
df_train.head()


,property_key,contractDate,price
0,p-c1ce31019,2018-03-01,1850000.0
1,p-a7fd7deb5,2018-03-01,1030000.0
2,p-2a981615e,2018-03-01,4300000.0
3,p-2bb570b5d,2018-03-01,1400888.0
4,p-4ccd6e1db,2018-03-01,725000.0


## Process macro and micro features


### Macro

Make all monthly to join on date

In [10]:
def process_date(df):
    df["date"] = pd.to_datetime(df["Data Series"], format="%Y %b ").dt.to_period("M")
    df = df.drop(columns="Data Series")
    df["date"] = df["date"].dt.to_timestamp()
    return df


In [11]:
df_cpi = process_date(df_cpi)
df_interest = process_date(df_interest)


In [12]:
def process_quarter(df):
    df["date"] = pd.to_datetime(
        df["Data Series"].str.replace(r"(\d+) (\d)Q ", r"\1-Q\2", regex=True)
    ).dt.to_period("M")
    df = df.set_index("date").resample("M", convention="end").interpolate("linear")
    df = df.reset_index()
    df = df.drop(columns="Data Series")
    df["date"] = df["date"].dt.to_timestamp()
    return df


In [13]:
df_rent_index = process_quarter(df_rent_index)
df_vacant = process_quarter(df_vacant)


C:\Users\Dante\AppData\Local\Temp\ipykernel_24224\1951963284.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(
C:\Users\Dante\AppData\Local\Temp\ipykernel_24224\1951963284.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(


### Micro

#### properties.csv

In [14]:
df_properties.head()


,area,floorRange,propertyType,district,typeOfArea,tenure,street,project,marketSegment,property_key
0,226.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-7b6c69000
1,194.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-320ed2726
2,348.0,06-10,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-2bc5c4951
3,223.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-f060c5be0
4,195.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-a8d1004a9


In [15]:
df_properties.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31530 entries, 0 to 31529
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   area           31530 non-null  float64
 1   floorRange     31530 non-null  object 
 2   propertyType   31530 non-null  object 
 3   district       31530 non-null  int64  
 4   typeOfArea     31530 non-null  object 
 5   tenure         31530 non-null  object 
 6   street         31530 non-null  object 
 7   project        31530 non-null  object 
 8   marketSegment  31530 non-null  object 
 9   property_key   31530 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 2.4+ MB


In [16]:
df_properties.groupby("floorRange")["property_key"].agg("count")


floorRange
-         4957
01-05    12037
06-10     5693
11-15     3912
16-20     2375
21-25     1052
26-30      662
31-35      411
36-40      200
41-45       94
46-50       45
51-55       32
56-60       22
61-65       18
66-70        5
71-75        1
B1-B5       14
Name: property_key, dtype: int64

In [17]:
df_properties.groupby("propertyType")["property_key"].agg("count")


propertyType
Apartment                 9782
Condominium              14881
Detached                   438
Executive Condominium     1946
Semi-detached             1425
Strata Detached             88
Strata Semi-detached       183
Strata Terrace             470
Terrace                   2317
Name: property_key, dtype: int64

In [18]:
df_properties.groupby("district")["property_key"].agg("count")


district
1      529
2      354
3      852
4      728
5     1228
6        8
7      299
8      544
9     1984
10    2548
11    1394
12    1081
13     723
14    1524
15    3123
16    1565
17     587
18    1348
19    3627
20    1221
21    1126
22     765
23    1675
25     427
26     425
27     866
28     979
Name: property_key, dtype: int64

In [19]:
df_properties.groupby("typeOfArea")["property_key"].agg("count")


typeOfArea
Land       4191
Strata    27339
Name: property_key, dtype: int64

In [20]:
df_properties.groupby("tenure")["property_key"].agg("count")


tenure
100 yrs lease commencing from 1986          11
102 yrs lease commencing from 1978          20
102 yrs lease commencing from 1996          19
103 yrs lease commencing from 1974           4
103 yrs lease commencing from 1975           6
                                         ...  
999999 yrs lease commencing from 1958        1
999999 yrs lease commencing from 1963        1
999999 yrs lease commencing from 1990        1
999999 yrs lease commencing from 1993        1
Freehold                                 14723
Name: property_key, Length: 122, dtype: int64

In [21]:
df_properties.groupby("street")["property_key"].agg("count")


street
ADAM ROAD           21
ADIS ROAD           15
AH HOOD ROAD        28
AH SOO GARDEN        3
AIDA STREET        125
                  ... 
YUK TONG AVENUE      5
YUNNAN CRESCENT     16
ZEHNDER ROAD         4
ZION CLOSE          12
ZION ROAD           15
Name: property_key, Length: 1171, dtype: int64

In [22]:
df_properties.groupby("project")["property_key"].agg("count")


project
# 1 LOFT               8
# 1 SUITES            12
1 CANBERRA            56
1 KING ALBERT PARK    12
1 MOULMEIN RISE        7
                      ..
YUEN SING MANSION      1
YUNNAN GARDENS        16
ZEDGE                 20
ZENITH                15
ZEPHYR PARK            2
Name: property_key, Length: 2753, dtype: int64

In [23]:
df_properties.groupby("marketSegment")["property_key"].agg("count")


marketSegment
CCR     7002
OCR    15737
RCR     8791
Name: property_key, dtype: int64

- floorRange: one hot encode
- propertyType: one hot encode
- district: to join with df_geo_attributes
- typeOfArea: one hot encode
- tenure: set to Freehold or non-Freehold (Lease)
- street: to join with df_geo_attributes
- project: to join with df_geo_attributes
- marketSegment: one hot encode

In [24]:
def encode_tenure(row):
    # Change to either Freehold or Lease
    tenure = row["tenure"]
    if tenure == "Freehold":
        row["tenure"] = 1
    else:
        row["tenure"] = 0
    return row


df_properties_encode = df_properties.apply(encode_tenure, axis=1)


In [25]:
df_properties_encode.groupby("tenure")["property_key"].agg("count")


tenure
0    16807
1    14723
Name: property_key, dtype: int64

In [26]:
# one hot encode for df_properties
def one_hot(col, df):
    ohe = OneHotEncoder()
    transformed = ohe.fit_transform(df[[col]])
    df[ohe.categories_[0]] = transformed.toarray()
    df = df.drop(columns=col)

    return df


col_names = ["floorRange", "propertyType", "typeOfArea", "marketSegment"]
count = 0

for col in col_names:
    if count == 0:
        df_properties_one_hot_encoded = one_hot(col, df_properties_encode)
        count += 1
    else:
        df_properties_one_hot_encoded = one_hot(col, df_properties_one_hot_encoded)


In [27]:
df_properties_one_hot_encoded.head()


,area,district,tenure,street,project,property_key,-,01-05,06-10,11-15,...,Semi-detached,Strata Detached,Strata Semi-detached,Strata Terrace,Terrace,Land,Strata,CCR,OCR,RCR
0,226.0,4,0,COVE DRIVE,TURQUOISE,p-7b6c69000,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,194.0,4,0,COVE DRIVE,TURQUOISE,p-320ed2726,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,348.0,4,0,COVE DRIVE,TURQUOISE,p-2bc5c4951,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,223.0,4,0,COVE DRIVE,TURQUOISE,p-f060c5be0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,195.0,4,0,COVE DRIVE,TURQUOISE,p-a8d1004a9,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


#### geo_attributes.csv

In [28]:
df_geo_attributes.head()


,street,project,district,lat,lng,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m
0,LIM AH WOO ROAD,SUITES @ GUILLEMARD,15,1.312520,103.891760,7,2,3
1,HOLLAND ROAD,LOFT@HOLLAND,10,1.312364,103.797169,0,3,4
2,SOUTH BUONA VISTA ROAD,VIVA VISTA,5,1.279803,103.785910,0,0,2
3,UPPER SERANGOON ROAD,PARK RESIDENCES KOVAN,19,1.357934,103.881932,7,2,1
4,NATHAN ROAD,LOFT @ NATHAN,10,1.294461,103.827621,2,4,4


- street: to join with df_geo_attributes
- project: to join with df_geo_attributes
- district: to join with df_geo_attributes
- lat: do not use
- lng: do not use
- num_schools_1km: no change
- num_supermarkets_500m: no change
- num_mrt_stations_500m: no change

## Model Training

### Combine dataframes

In [29]:
# Join df_properties and df_geo_attributes
df_micro_encoded = pd.merge(
    df_properties_one_hot_encoded,
    df_geo_attributes,
    on=["street", "project", "district"],
    how="outer",
)
df_micro_encoded = df_micro_encoded.drop(columns=["street", "project", "district"])

# Drop lat and long
df_micro_encoded = df_micro_encoded.drop(columns=["lat", "lng"])


In [30]:
df_micro_encoded.head()


,area,tenure,property_key,-,01-05,06-10,11-15,16-20,21-25,26-30,...,Strata Terrace,Terrace,Land,Strata,CCR,OCR,RCR,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m
0,226.0,0,p-7b6c69000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,194.0,0,p-320ed2726,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,348.0,0,p-2bc5c4951,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,223.0,0,p-f060c5be0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,195.0,0,p-a8d1004a9,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [31]:
df_macro = pd.merge(df_cpi, df_interest, on="date", how="outer")
df_macro = pd.merge(df_macro, df_rent_index, on="date", how="outer")
df_macro = pd.merge(df_macro, df_vacant, on="date", how="outer")


In [32]:
# Fill empty with latest data
df_macro = df_macro.fillna(method="bfill")


In [33]:
df_macro.head()


,CPI,date,InterestRate,RentIndex,Available,Vacant
0,111.186,2022-12-01,1.48,148.1,34084.0,524.000000
1,110.959,2022-11-01,1.47,148.1,34084.0,524.000000
2,109.893,2022-10-01,1.17,148.1,34084.0,524.000000
3,110.339,2022-09-01,1.03,144.7,34084.0,520.666667
4,109.863,2022-08-01,0.95,141.3,34084.0,517.333333


In [34]:
df_train_encoded = df_train.rename(columns={"contractDate": "date"})
df_train_encoded["date"] = pd.to_datetime(df_train_encoded["date"])


In [35]:
df_train_encoded = pd.merge(df_train_encoded, df_micro_encoded, on="property_key", how="left")
df_train_encoded = pd.merge(df_train_encoded, df_macro, on="date", how="left")


In [36]:
df_train_encoded = df_train_encoded.set_index("property_key")


In [37]:
df_train_encoded.head()


,date,price,area,tenure,-,01-05,06-10,11-15,16-20,21-25,...,OCR,RCR,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,CPI,InterestRate,RentIndex,Available,Vacant
property_key,,,,,,,,,,,,,,,,,,,,,
p-c1ce31019,2018-03-01,1850000.0,102.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,13.0,99.299,0.5,103.466667,31095.0,4320.0
p-a7fd7deb5,2018-03-01,1030000.0,121.0,1,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,1.0,0.0,99.299,0.5,103.466667,31095.0,4320.0
p-2a981615e,2018-03-01,4300000.0,300.0,1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,1.0,99.299,0.5,103.466667,31095.0,4320.0
p-2bb570b5d,2018-03-01,1400888.0,107.0,1,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,6.0,0.0,2.0,99.299,0.5,103.466667,31095.0,4320.0
p-4ccd6e1db,2018-03-01,725000.0,54.0,0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,1.0,99.299,0.5,103.466667,31095.0,4320.0


In [38]:
df_train_encoded.loc[df_train_encoded.isnull().any(axis=1)]


,date,price,area,tenure,-,01-05,06-10,11-15,16-20,21-25,...,OCR,RCR,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,CPI,InterestRate,RentIndex,Available,Vacant
property_key,,,,,,,,,,,,,,,,,,,,,
p-53b215525,2020-09-01,8880000.0,923.1,1,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,NaN,NaN,NaN,100.139,0.25,103.866667,33874.666667,1026.333333


In [39]:
df_train_encoded = df_train_encoded.drop(index="p-53b215525")


In [40]:
df_train_encoded.loc[df_train_encoded.isnull().any(axis=1)]


,date,price,area,tenure,-,01-05,06-10,11-15,16-20,21-25,...,OCR,RCR,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,CPI,InterestRate,RentIndex,Available,Vacant
property_key,,,,,,,,,,,,,,,,,,,,,


#### Feature Engineering

In [41]:
df_train_encoded.info()


<class 'pandas.core.frame.DataFrame'>
Index: 62948 entries, p-c1ce31019 to p-7424f0b63
Data columns (total 43 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   62948 non-null  datetime64[ns]
 1   price                  62948 non-null  float64       
 2   area                   62948 non-null  float64       
 3   tenure                 62948 non-null  int64         
 4   -                      62948 non-null  float64       
 5   01-05                  62948 non-null  float64       
 6   06-10                  62948 non-null  float64       
 7   11-15                  62948 non-null  float64       
 8   16-20                  62948 non-null  float64       
 9   21-25                  62948 non-null  float64       
 10  26-30                  62948 non-null  float64       
 11  31-35                  62948 non-null  float64       
 12  36-40                  62948 non-null  float64   

In [42]:
def create_new_features(df):
    df["area^2"] = df["area"] ** 2
    df["tenure^2"] = df["tenure"] ** 2
    df["num_schools_1km^2"] = df["num_schools_1km"] ** 2
    df["num_supermarkets_500m^2"] = df["num_supermarkets_500m"] ** 2
    df["num_mrt_stations_500m^2"] = df["num_mrt_stations_500m"] ** 2
    df["CPI^2"] = df["CPI"] ** 2
    df["InterestRate^2"] = df["InterestRate"] ** 2
    df["RentIndex^2"] = df["RentIndex"] ** 2
    df["Available^2"] = df["Available"] ** 2
    df["Vacant^2"] = df["Vacant"] ** 2

    return df


In [43]:
df_train_encoded = create_new_features(df_train_encoded)

In [44]:
df_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62948 entries, p-c1ce31019 to p-7424f0b63
Data columns (total 53 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     62948 non-null  datetime64[ns]
 1   price                    62948 non-null  float64       
 2   area                     62948 non-null  float64       
 3   tenure                   62948 non-null  int64         
 4   -                        62948 non-null  float64       
 5   01-05                    62948 non-null  float64       
 6   06-10                    62948 non-null  float64       
 7   11-15                    62948 non-null  float64       
 8   16-20                    62948 non-null  float64       
 9   21-25                    62948 non-null  float64       
 10  26-30                    62948 non-null  float64       
 11  31-35                    62948 non-null  float64       
 12  36-40                

In [45]:
y = df_train_encoded["price"]
X = df_train_encoded.drop(columns="price")


# Drop date for now, as linear regression model cannot use datetime
X = X.drop(columns="date")


### Linear Regression Model

In [46]:
lin_reg = LinearRegression()
lin_reg.fit(X, y)


LinearRegression()

### Submission

In [47]:
df_macro.head()


,CPI,date,InterestRate,RentIndex,Available,Vacant
0,111.186,2022-12-01,1.48,148.1,34084.0,524.000000
1,110.959,2022-11-01,1.47,148.1,34084.0,524.000000
2,109.893,2022-10-01,1.17,148.1,34084.0,524.000000
3,110.339,2022-09-01,1.03,144.7,34084.0,520.666667
4,109.863,2022-08-01,0.95,141.3,34084.0,517.333333


In [48]:
df_new_months = pd.DataFrame({"date": ["2023-03-01", "2023-02-01", "2023-01-01"]})
df_new_months["date"] = pd.to_datetime(df_new_months["date"])


In [49]:
df_new_months.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 152.0 bytes


In [50]:
df_macro = pd.concat([df_new_months, df_macro])


In [51]:
df_macro = df_macro.fillna(method="bfill")


In [52]:
df_macro.head()


,date,CPI,InterestRate,RentIndex,Available,Vacant
0,2023-03-01,111.186,1.48,148.1,34084.0,524.0
1,2023-02-01,111.186,1.48,148.1,34084.0,524.0
2,2023-01-01,111.186,1.48,148.1,34084.0,524.0
0,2022-12-01,111.186,1.48,148.1,34084.0,524.0
1,2022-11-01,110.959,1.47,148.1,34084.0,524.0


In [53]:
df_val = df_test


In [54]:
df_val = df_val.rename(columns={"contractDate": "date"})
df_val["date"] = pd.to_datetime(df_val["date"])


In [55]:
df_val = pd.merge(df_val, df_micro_encoded, on="property_key", how="left")
df_val = pd.merge(df_val, df_macro, on="date", how="left")


In [56]:
df_val = df_val.set_index("property_key")


In [57]:
df_val.head()


,date,area,tenure,-,01-05,06-10,11-15,16-20,21-25,26-30,...,OCR,RCR,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,CPI,InterestRate,RentIndex,Available,Vacant
property_key,,,,,,,,,,,,,,,,,,,,,
p-ff93e87ab,2023-01-01,147.0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,1.0,111.186,1.48,148.1,34084.0,524.0
p-38fe6afe9,2023-01-01,99.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,0.0,1.0,111.186,1.48,148.1,34084.0,524.0
p-fc9650179,2023-01-01,99.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,6.0,0.0,1.0,111.186,1.48,148.1,34084.0,524.0
p-59a09ad08,2023-01-01,108.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,111.186,1.48,148.1,34084.0,524.0
p-808332e5c,2023-01-01,72.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,6.0,1.0,111.186,1.48,148.1,34084.0,524.0


In [58]:
df_val.loc[df_val.isnull().any(axis=1)]


,date,area,tenure,-,01-05,06-10,11-15,16-20,21-25,26-30,...,OCR,RCR,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,CPI,InterestRate,RentIndex,Available,Vacant
property_key,,,,,,,,,,,,,,,,,,,,,
p-f4822a16f,2023-03-01,939.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,NaN,NaN,NaN,111.186,1.48,148.1,34084.0,524.0


In [59]:
df_val["num_schools_1km"] = df_val["num_schools_1km"].fillna(df_val["num_schools_1km"].median())
df_val["num_supermarkets_500m"] = df_val["num_supermarkets_500m"].fillna(
    df_val["num_supermarkets_500m"].median()
)
df_val["num_mrt_stations_500m"] = df_val["num_mrt_stations_500m"].fillna(
    df_val["num_mrt_stations_500m"].median()
)


In [60]:
# df_val = df_val.drop(index="p-f4822a16f")


In [61]:
df_val.loc[df_val.isnull().any(axis=1)]


,date,area,tenure,-,01-05,06-10,11-15,16-20,21-25,26-30,...,OCR,RCR,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,CPI,InterestRate,RentIndex,Available,Vacant
property_key,,,,,,,,,,,,,,,,,,,,,


In [62]:
df_val = create_new_features(df_val)

In [63]:
# Drop date for now, as linear regression model cannot use datetime
X_val = df_val.drop(columns="date")


In [64]:
X_val


,area,tenure,-,01-05,06-10,11-15,16-20,21-25,26-30,31-35,...,area^2,tenure^2,num_schools_1km^2,num_supermarkets_500m^2,num_mrt_stations_500m^2,CPI^2,InterestRate^2,RentIndex^2,Available^2,Vacant^2
property_key,,,,,,,,,,,,,,,,,,,,,
p-ff93e87ab,147.0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21609.00,1,4.0,0.0,1.0,12362.326596,2.1904,21933.61,1.161719e+09,274576.0
p-38fe6afe9,99.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,9801.00,0,4.0,0.0,1.0,12362.326596,2.1904,21933.61,1.161719e+09,274576.0
p-fc9650179,99.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,9801.00,0,36.0,0.0,1.0,12362.326596,2.1904,21933.61,1.161719e+09,274576.0
p-59a09ad08,108.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11664.00,0,0.0,0.0,1.0,12362.326596,2.1904,21933.61,1.161719e+09,274576.0
p-808332e5c,72.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,5184.00,0,25.0,36.0,1.0,12362.326596,2.1904,21933.61,1.161719e+09,274576.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
p-56681c564,84.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,7056.00,0,9.0,4.0,1.0,12362.326596,2.1904,21933.61,1.161719e+09,274576.0
p-89a6ff5de,290.5,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,84390.25,1,36.0,16.0,49.0,12362.326596,2.1904,21933.61,1.161719e+09,274576.0
p-920d750e9,115.0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,13225.00,1,36.0,25.0,1.0,12362.326596,2.1904,21933.61,1.161719e+09,274576.0


In [65]:
results = lin_reg.predict(X_val)


In [66]:
df_results = pd.Series(results, name="prediction")


In [67]:
df_results


0       3.455124e+06
1       1.645032e+06
2       1.276541e+06
3       1.531005e+06
4       1.046701e+06
            ...     
2326    1.307427e+06
2327    5.992399e+06
2328    3.117839e+06
2329    2.776701e+06
2330    9.053213e+05
Name: prediction, Length: 2331, dtype: float64

In [68]:
df_test


,property_key,contractDate
0,p-ff93e87ab,2023-01-01
1,p-38fe6afe9,2023-01-01
2,p-fc9650179,2023-01-01
3,p-59a09ad08,2023-01-01
4,p-808332e5c,2023-01-01
...,...,...
2326,p-56681c564,2023-03-01
2327,p-89a6ff5de,2023-03-01
2328,p-920d750e9,2023-03-01
2329,p-e7146b43c,2023-03-01


In [69]:
df_submission = pd.concat([df_test, df_results], axis=1)


In [70]:
df_submission.head()


,property_key,contractDate,prediction
0,p-ff93e87ab,2023-01-01,3.455124e+06
1,p-38fe6afe9,2023-01-01,1.645032e+06
2,p-fc9650179,2023-01-01,1.276541e+06
3,p-59a09ad08,2023-01-01,1.531005e+06
4,p-808332e5c,2023-01-01,1.046701e+06


In [71]:
df_submission.to_csv("data/linear_reg_submission.csv", index=False)


### XGBoost

In [72]:
# Join df_properties and df_geo_attributes
df_micro = pd.merge(
    df_properties, df_geo_attributes, on=["street", "project", "district"], how="outer"
)
df_micro = df_micro.drop(columns=["street", "project", "district"])

# Drop lat and long
df_micro = df_micro.drop(columns=["lat", "lng"])


In [73]:
df_macro = pd.merge(df_cpi, df_interest, on="date", how="outer")
df_macro = pd.merge(df_macro, df_rent_index, on="date", how="outer")
df_macro = pd.merge(df_macro, df_vacant, on="date", how="outer")


In [74]:
# Fill empty with latest data
df_macro = df_macro.fillna(method="bfill")


In [75]:
df_train = df_train.rename(columns={"contractDate": "date"})
df_train["date"] = pd.to_datetime(df_train["date"])


In [76]:
df_train = pd.merge(df_train, df_micro, on="property_key", how="left")
df_train = pd.merge(df_train, df_macro, on="date", how="left")


In [77]:
df_train = df_train.set_index("property_key")


In [78]:
df_train.loc[df_train.isnull().any(axis=1)]


,date,price,area,floorRange,propertyType,typeOfArea,tenure,marketSegment,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,CPI,InterestRate,RentIndex,Available,Vacant
property_key,,,,,,,,,,,,,,,,
p-53b215525,2020-09-01,8880000.0,923.1,-,Detached,Land,Freehold,OCR,NaN,NaN,NaN,100.139,0.25,103.866667,33874.666667,1026.333333


In [79]:
cat_features = ["floorRange", "propertyType", "typeOfArea", "marketSegment", "tenure"]
for cat in cat_features:
    df_train[cat] = df_train[cat].astype("category")


In [80]:
y = df_train["price"]
X = df_train.drop(columns="price")


# Drop date for now, as linear regression model cannot use datetime
X = X.drop(columns="date")


In [81]:
X.info()


<class 'pandas.core.frame.DataFrame'>
Index: 62949 entries, p-c1ce31019 to p-7424f0b63
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   area                   62949 non-null  float64 
 1   floorRange             62949 non-null  category
 2   propertyType           62949 non-null  category
 3   typeOfArea             62949 non-null  category
 4   tenure                 62949 non-null  category
 5   marketSegment          62949 non-null  category
 6   num_schools_1km        62948 non-null  float64 
 7   num_supermarkets_500m  62948 non-null  float64 
 8   num_mrt_stations_500m  62948 non-null  float64 
 9   CPI                    62949 non-null  float64 
 10  InterestRate           62949 non-null  float64 
 11  RentIndex              62949 non-null  float64 
 12  Available              62949 non-null  float64 
 13  Vacant                 62949 non-null  float64 
dtypes: category(5), float64(9)


In [82]:
xgb = XGBRegressor(
    booster="gbtree",
    eval_metric="rmse",
    tree_method="gpu_hist",
    colsample_bylevel=1,
    colsample_bynode=1,
    colsample_bytree=1,
    gamma=0,
    importance_type="gain",
    learning_rate=0.3,
    max_delta_step=0,
    max_depth=4,
    min_child_weight=1,
    n_estimators=1000,
    nthread=4,
    objective="reg:linear",
    random_state=42,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=42,
    subsample=0.5,
    sampling_method="uniform",
    verbosity=1,
    enable_categorical=True,
)
xgb.fit(X, y)


[05:26:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=True,
             eval_metric='rmse', feature_types=None, gamma=0, gpu_id=None,
             grow_policy=None, importance_type='gain',
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=0,
             max_depth=4, max_leaves=None, min_child_weight=1, missing=nan,
             monotone_constraints=None, n_estimators=1000, n_jobs=None,
             nthread=4, num_parallel_tree=None, objective='reg:linear', ...)

### Submission

In [83]:
df_macro.head()


,CPI,date,InterestRate,RentIndex,Available,Vacant
0,111.186,2022-12-01,1.48,148.1,34084.0,524.000000
1,110.959,2022-11-01,1.47,148.1,34084.0,524.000000
2,109.893,2022-10-01,1.17,148.1,34084.0,524.000000
3,110.339,2022-09-01,1.03,144.7,34084.0,520.666667
4,109.863,2022-08-01,0.95,141.3,34084.0,517.333333


In [84]:
df_new_months = pd.DataFrame({"date": ["2023-03-01", "2023-02-01", "2023-01-01"]})
df_new_months["date"] = pd.to_datetime(df_new_months["date"])


In [85]:
df_new_months.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 152.0 bytes


In [86]:
df_macro = pd.concat([df_new_months, df_macro])


In [87]:
df_macro = df_macro.fillna(method="bfill")


In [88]:
df_macro.head()


,date,CPI,InterestRate,RentIndex,Available,Vacant
0,2023-03-01,111.186,1.48,148.1,34084.0,524.0
1,2023-02-01,111.186,1.48,148.1,34084.0,524.0
2,2023-01-01,111.186,1.48,148.1,34084.0,524.0
0,2022-12-01,111.186,1.48,148.1,34084.0,524.0
1,2022-11-01,110.959,1.47,148.1,34084.0,524.0


In [89]:
df_val = df_test


In [90]:
df_val = df_val.rename(columns={"contractDate": "date"})
df_val["date"] = pd.to_datetime(df_val["date"])


In [91]:
df_val = pd.merge(df_val, df_micro, on="property_key", how="left")
df_val = pd.merge(df_val, df_macro, on="date", how="left")


In [92]:
df_val = df_val.set_index("property_key")


In [93]:
df_val.head()


,date,area,floorRange,propertyType,typeOfArea,tenure,marketSegment,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,CPI,InterestRate,RentIndex,Available,Vacant
property_key,,,,,,,,,,,,,,,
p-ff93e87ab,2023-01-01,147.0,01-05,Condominium,Strata,Freehold,CCR,2.0,0.0,1.0,111.186,1.48,148.1,34084.0,524.0
p-38fe6afe9,2023-01-01,99.0,06-10,Condominium,Strata,99 yrs lease commencing from 1991,RCR,2.0,0.0,1.0,111.186,1.48,148.1,34084.0,524.0
p-fc9650179,2023-01-01,99.0,06-10,Condominium,Strata,99 yrs lease commencing from 1995,OCR,6.0,0.0,1.0,111.186,1.48,148.1,34084.0,524.0
p-59a09ad08,2023-01-01,108.0,01-05,Condominium,Strata,99 yrs lease commencing from 2000,OCR,0.0,0.0,1.0,111.186,1.48,148.1,34084.0,524.0
p-808332e5c,2023-01-01,72.0,06-10,Condominium,Strata,99 yrs lease commencing from 2011,OCR,5.0,6.0,1.0,111.186,1.48,148.1,34084.0,524.0


In [94]:
df_val.loc[df_val.isnull().any(axis=1)]


,date,area,floorRange,propertyType,typeOfArea,tenure,marketSegment,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,CPI,InterestRate,RentIndex,Available,Vacant
property_key,,,,,,,,,,,,,,,
p-f4822a16f,2023-03-01,939.0,-,Strata Detached,Strata,Freehold,OCR,NaN,NaN,NaN,111.186,1.48,148.1,34084.0,524.0


In [95]:
df_val["num_schools_1km"] = df_val["num_schools_1km"].fillna(df_val["num_schools_1km"].median())
df_val["num_supermarkets_500m"] = df_val["num_supermarkets_500m"].fillna(
    df_val["num_supermarkets_500m"].median()
)
df_val["num_mrt_stations_500m"] = df_val["num_mrt_stations_500m"].fillna(
    df_val["num_mrt_stations_500m"].median()
)


In [96]:
df_val.loc[df_val.isnull().any(axis=1)]


,date,area,floorRange,propertyType,typeOfArea,tenure,marketSegment,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m,CPI,InterestRate,RentIndex,Available,Vacant
property_key,,,,,,,,,,,,,,,


In [97]:
cat_features = ["floorRange", "propertyType", "typeOfArea", "marketSegment", "tenure"]
for cat in cat_features:
    df_val[cat] = df_val[cat].astype("category")


In [98]:
# Drop date for now, as linear regression model cannot use datetime
X_val = df_val.drop(columns="date")


In [99]:
results = xgb.predict(X_val)


In [100]:
df_results = pd.Series(results, name="prediction")


In [101]:
df_submission = pd.concat([df_test, df_results], axis=1)


In [102]:
df_submission.to_csv("data/xgb_submission.csv", index=False)
